In [1]:
import pandas as pd
import numpy as np
import ast
import os

In [ ]:


# Paths (change if your folder names differ)
raw_battles_path = r"C:\DSC106\project4\CR_data\BattlesStaging_small.csv"
card_master_path = r"C:\DSC106\project4\CR_data\CardMasterListSeason18_12082020_with_elixir_and_rarity.csv"

# Output file for the new dataset
output_path = r"C:\DSC106\project4\CR_data\BattlesStaging_with_card_features.csv"


Load card metadata and build lookup dictionaries

In [3]:
# Read the card metadata (with type, elixir, rarity)
cards_meta = pd.read_csv(card_master_path)

# Use card ID as index so we can look things up quickly
cards_meta = cards_meta.set_index("team.card1.id")

# Build dictionaries: card ID -> attribute
id_to_name   = cards_meta["team.card1.name"].to_dict()
id_to_type   = cards_meta["card_type"].to_dict()   # column is "card_type" in this file
id_to_elixir = cards_meta["elixir"].to_dict()
id_to_rarity = cards_meta["rarity"].to_dict()

# Quick sanity check
list(id_to_name.items())[:5]


[(26000000, 'Knight'),
 (26000001, 'Archers'),
 (26000002, 'Goblins'),
 (26000003, 'Giant'),
 (26000004, 'P.E.K.K.A')]

Helper function to expand one side (winner/loser)

In [4]:
def add_card_features(df_chunk, side, list_col):
    """
    side: "winner" or "loser"
    list_col: column name with the list of card IDs
              e.g. "winner.cards.list" or "loser.cards.list"
    """
    def parse_list(x):
        # x is a string like "[26000036, 26000043, ...]"
        if isinstance(x, str) and x.strip():
            try:
                lst = ast.literal_eval(x)
                # Make sure length is exactly 8
                if len(lst) < 8:
                    lst = lst + [np.nan] * (8 - len(lst))
                elif len(lst) > 8:
                    lst = lst[:8]
                return lst
            except Exception:
                return [np.nan] * 8
        else:
            return [np.nan] * 8

    # Turn the list column into 8 separate columns of card IDs
    ids_df = df_chunk[list_col].apply(parse_list).apply(pd.Series)

    # For slot 1..8, map ID to name/type/elixir/rarity
    for i in range(8):
        card_ids = ids_df[i]
        df_chunk[f"{side}_card_{i+1}_name"]   = card_ids.map(id_to_name)
        df_chunk[f"{side}_card_{i+1}_type"]   = card_ids.map(id_to_type)
        df_chunk[f"{side}_card_{i+1}_elixir"] = card_ids.map(id_to_elixir)
        df_chunk[f"{side}_card_{i+1}_rarity"] = card_ids.map(id_to_rarity)

    return df_chunk


Load raw battles, add winner + loser features, reorder columns, save

In [ ]:
# Read the raw dataset with winner + loser card lists
battles = pd.read_csv(raw_battles_path)

print("Raw shape:", battles.shape)
print(battles.head())

# Add winner card features
battles = add_card_features(battles, side="winner", list_col="winner.cards.list")

# Add loser card features
battles = add_card_features(battles, side="loser", list_col="loser.cards.list")

# ----- Reorder columns -----

# Keep the original four columns first
base_cols = [
    "battleTime",
    "average.startingTrophies",
    "winner.cards.list",
    "loser.cards.list",
]

# Then winner_card_1_name, winner_card_1_type, winner_card_1_elixir, winner_card_1_rarity, ...
winner_cols = []
for i in range(1, 9):
    winner_cols.extend([
        f"winner_card_{i}_name",
        f"winner_card_{i}_type",
        f"winner_card_{i}_elixir",
        f"winner_card_{i}_rarity",
    ])

# Then loser_card_1_name, loser_card_1_type, loser_card_1_elixir, loser_card_1_rarity, ...
loser_cols = []
for i in range(1, 9):
    loser_cols.extend([
        f"loser_card_{i}_name",
        f"loser_card_{i}_type",
        f"loser_card_{i}_elixir",
        f"loser_card_{i}_rarity",
    ])

new_order = base_cols + winner_cols + loser_cols

# Select columns in this exact order
battles_final = battles[new_order]

print("Final shape:", battles_final.shape)
battles_final.head()


Raw shape: (37973387, 4)
                  battleTime  average.startingTrophies  \
0  2020-12-07 07:00:00+00:00                    6590.0   
1  2020-12-07 07:00:00+00:00                    5582.5   
2  2020-12-07 07:00:02+00:00                    5684.0   
3  2020-12-07 07:00:03+00:00                    6031.0   
4  2020-12-07 07:00:06+00:00                    5140.0   

                                   winner.cards.list  \
0  [26000036, 26000043, 26000044, 26000050, 26000...   
1  [26000012, 26000017, 26000022, 26000024, 26000...   
2  [26000004, 26000036, 26000042, 26000046, 26000...   
3  [26000004, 26000006, 26000007, 26000011, 26000...   
4  [26000004, 26000011, 26000013, 26000022, 26000...   

                                    loser.cards.list  
0  [26000000, 26000026, 26000030, 26000041, 27000...  
1  [26000000, 26000003, 26000007, 26000011, 26000...  
2  [26000011, 26000026, 26000030, 26000041, 27000...  
3  [26000032, 26000040, 26000041, 26000049, 26000...  
4  [26000012, 

Save the new dataset

In [4]:
df0 = pd.read_csv("Cards0")
df1 = pd.read_csv("Cards1")
df2 = pd.read_csv("Cards2")
df3 = pd.read_csv("Cards3")
df4 = pd.read_csv("Cards4")

In [7]:
df0 = df0.rename(columns={'count':'count_0'})
df0['count_1'] = df1['count']
df0['count_2'] = df2['count']
df0['count_3'] = df3['count']
df0['count_4'] = df4['count']

In [13]:
df0['overall_count'] = df0['count_0'] + df0['count_1'] + df0['count_2'] + df0['count_3'] + df0['count_4']
df0

,team.card1.id,team.card1.name,card_type,elixir,rarity,count_0,count_1,count_2,count_3,count_4,overall_count
0,26000000,Knight,troop,3,Common,24614.0,90222.0,95205.0,79899.0,64519.0,354459.0
1,26000001,Archers,troop,3,Common,8373.0,29171.0,38557.0,23334.0,13438.0,112873.0
2,26000002,Goblins,troop,2,Common,1511.0,4028.0,6113.0,5567.0,3905.0,21124.0
3,26000003,Giant,troop,5,Rare,5000.0,18537.0,41360.0,24764.0,15268.0,104929.0
4,26000004,P.E.K.K.A,troop,7,Epic,22460.0,56020.0,68813.0,56357.0,27577.0,231227.0
...,...,...,...,...,...,...,...,...,...,...,...
97,28000014,Earthquake,spell,3,Rare,1499.0,4847.0,8091.0,14746.0,15786.0,44969.0
98,28000015,Barbarian Barrel,spell,2,Epic,10870.0,26882.0,29239.0,55042.0,67916.0,189949.0
99,28000016,Heal Spirit,spell,1,Rare,2096.0,5696.0,4380.0,4937.0,4858.0,21967.0
100,28000017,Giant Snowball,spell,2,Common,3004.0,8997.0,8636.0,14427.0,20719.0,55783.0


In [14]:
df0.to_csv("CardList.csv", index=False)

